Okey here is explained what i have doned. I have used pandas for storing the results.

The structure of the information: '.sqlite' files with one table named 'data' that contains all the tweets. The tweets has:
    - Index, unique id of the tweet.
    - Language, language of the tweet.
    - Location, location of the user according to his profile information.
    - List of the candidates (Arthaud, Asselineau, Cheminade, Dupont-Aignan, Filon, Hamon, Lassalle, Le-Pen, Macron, Melenchon, Poutou). The candidates have a 1 if the tweet is referred to them.
    - Original Tweet, the original tweet if is a retweeted tweet or corresponds to someone answering the tweet.
    - Original User, user that has written the original tweet.
    - Retweeted, 1 or 0 depending if the tweets is retweeted or not.
    - Tweet, text
    - Timestamp in milliseconds of the hour that the tweet has been written
    - User, name of the user
    - Day
    - This parameter i didnt know what it is but i have deleted it


First i have read all the '.sqlites' file and i have filter to obtain only the ones that are written in English, that is the condition in the first if structure.

Moreover i have deleted some irrelevant data like the id of the tweet (because we have eliminate some of them), the language and the last paramater. I have pass the hour that was in miliseconds to normal datetime and renamed all the columns to have an idea of what structured our data is going to have.

I have used the vaderSentiment tool (you could install it via 'pip install vaderSentiment') and analyse the text to calculate the sentiment of each tweet. I have added this new column to the dataFrame and save the data in a .csv file.

In [1]:
import sqlite3 as lite

In [6]:
from os import scandir, getcwd

def ls(ruta = getcwd()+'/data/'):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [7]:
files = ls()
data_en = list()
all_data = list()
for file in files:
    con = lite.connect('data/'+file)

    cur = con.cursor()    
    cur.execute('SELECT * from data')

    data = cur.fetchall()
    for tweet in data:
        if tweet[1] == 'en':
            data_en.append(tweet)
        #all_data.append(tweet)

In [2]:
from geotext import GeoText

def extract_city_country(location):
    city = ''
    country = ''
    places = GeoText(location.lower().title().strip())
    if len(places.cities) != 0:
        city = places.cities[0]
        if len(list(places.country_mentions.items())):
            if places.cities[0] == "Paris":
                country = "FR"
            else: country = list(places.country_mentions.items())[0][0]
    else:
        if len(list(places.country_mentions.items())):
            country = list(places.country_mentions.items())[0][0]
    return city,country
            

In [8]:
data_final_en = list()

for tweet in data_en:
    if tweet[2]:
        city,country = extract_city_country(tweet[2])
        t = tweet + (city, country)
    else:
        t = tweet + ("","")

    data_final_en.append(t)

In [3]:
import pandas

In [9]:
df = pandas.DataFrame(data=data_final_en)

In [10]:
del df[0] #delete indexing

In [11]:
del df[1] #delete language because we have extracted only English tweets

In [12]:
import time

# pass timestamp in millisecond to datetime
df[18] = df[18].apply(lambda x: time.strftime("%D %H:%M", time.localtime(int(x)/1000)).split()[1])


In [13]:
del df[21] #delete

In [14]:
df.columns = ['Location', 'Anthaud', 'Asselineau', 'Cheminade', 'Dupont-Aignan', 'Fillon', 'Hamon', 'Lassalle', 'Le-Pen', 'Macron',
             'Melenchon', 'Poutou', 'Original-Tweet', 'Original-User', 'Retweeted', 'Tweet', 'Hour', 'User', 'Day', 'City', 'Country']


In [15]:
import re
def clean_jumplines(text):
    if text != None:
        text = re.sub('\n',' ',text)
        text = re.sub('\r',' ',text)
        text = re.sub('\t',' ',text)
        text = re.sub('\'',' ',text)
        text = re.sub('‘',' ',text)
        text = re.sub('’',' ',text)
        text = re.sub(';',' ',text)
    return text
    

In [16]:
df['Tweet'] = df['Tweet'].apply(clean_jumplines)
df['Original-Tweet'] = df['Original-Tweet'].apply(clean_jumplines)


In [17]:
df[df.columns] = df[df.columns].replace({',':' '}, regex=True)

In [18]:
del df['Location']

In [19]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
sentiment = list()

for tweet in df['Tweet']:
    sentiment.append(analyzer.polarity_scores(tweet))


In [20]:
real_sentiments = list()
compound = list()
for s in sentiment:
    if s['compound'] >= 0.5:
        real_sentiments.append('Positive')
    elif s['compound'] <= -0.5:
        real_sentiments.append('Negative')
    else:
        real_sentiments.append('Neutral')
    compound.append(s['compound'])

In [21]:
df['Sentiment'] = pandas.Series(real_sentiments)
df['Compound'] = pandas.Series(compound)

In [ ]:
writer = pandas.ExcelWriter('data-en.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [22]:
df.to_csv('elections_tweets_en.csv',sep=',')